In [1]:
import os
import sys
import numpy as np
import h5py
root = os.path.dirname(os.path.abspath(os.curdir))
sys.path.append(root)

import torch
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
from torch.utils.data import Subset

from predify.utils.training import train_pcoders, eval_pcoders

from networks_2022 import BranchedNetwork
from data.CleanSoundsDataset import CleanSoundsDataset, TrainCleanSoundsDataset, PsychophysicsCleanSoundsDataset

# Specify Network to train
TODO: This should be converted to a script that accepts arguments for which network to train

In [2]:
from pbranchednetwork_a1 import PBranchedNetwork_A1SeparateHP
PNetClass = PBranchedNetwork_A1SeparateHP
pnet_name = 'a1'

In [3]:
from pbranchednetwork_conv1 import PBranchedNetwork_Conv1SeparateHP
PNetClass = PBranchedNetwork_Conv1SeparateHP
pnet_name = 'conv1'

In [2]:
from pbranchednetwork_all import PBranchedNetwork_AllSeparateHP
PNetClass = PBranchedNetwork_AllSeparateHP
pnet_name = 'all'

# Parameters

In [3]:
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'Device: {DEVICE}')
BATCH_SIZE = 64
NUM_WORKERS = 2
PIN_MEMORY = True
NUM_EPOCHS = 50

lr = 1E-4
engram_dir = '/mnt/smb/locker/issa-locker/users/Erica/'
checkpoints_dir = f'{engram_dir}hcnn/checkpoints/'
tensorboard_dir = f'{engram_dir}hcnn/tensorboard/'
train_datafile = f'{engram_dir}seed_542_word_clean_random_order.hdf5'

Device: cuda:0


In [4]:
!nvidia-smi

Tue Jun 21 17:32:30 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.48.07    Driver Version: 515.48.07    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:B1:00.0 Off |                  N/A |
|  0%   27C    P8    11W / 250W |      3MiB / 11264MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Load network and optimizer

In [5]:
net = BranchedNetwork()
net.load_state_dict(torch.load(f'{engram_dir}networks_2022_weights.pt'))

/share/issa/users/es3773/hallucnn/src/models/layers.py:78: UserWarning: Inconsistent tf pad calculation in ConvLayer.
  warnings.warn('Inconsistent tf pad calculation in ConvLayer.')
/share/issa/users/es3773/hallucnn/src/models/layers.py:173: UserWarning: Inconsistent tf pad calculation: 0, 1
  warnings.warn(f'Inconsistent tf pad calculation: {pad_left}, {pad_right}')


<All keys matched successfully>

In [6]:
pnet = PNetClass(net, build_graph=True)

In [7]:
pnet.load_state_dict(torch.load(
    f"{checkpoints_dir}all/all-25-regular.pth",
    map_location='cpu'
    ))

<All keys matched successfully>

In [8]:
pnet.eval()

PBranchedNetwork_AllSeparateHP(
  (backbone): BranchedNetwork(
    (speech_branch): Sequential(
      (conv1): ConvLayer(
        (block): Sequential(
          (0): Conv2d(1, 96, kernel_size=(6, 14), stride=(3, 3), padding=(2, 6))
          (1): ReLU()
        )
      )
      (rnorm1): LRNorm(
        (block): LocalResponseNorm(5, alpha=0.005, beta=0.75, k=1.0)
      )
      (pool1): PoolLayer(
        (block): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
      )
      (conv2): ConvLayer(
        (block): Sequential(
          (0): Conv2d(96, 256, kernel_size=(5, 5), stride=(2, 2), padding=(1, 2))
          (1): ReLU()
        )
      )
      (rnorm2): LRNorm(
        (block): LocalResponseNorm(5, alpha=0.005, beta=0.75, k=1.0)
      )
      (pool2): PoolLayer(
        (block): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
      )
      (conv3): ConvLayer(
        (block): Sequential(
          (0): Conv2d(256, 512, kernel_size=

In [ ]:
pnet.to(DEVICE)
optimizer = torch.optim.Adam(
    [{'params':getattr(pnet,f"pcoder{x+1}").pmodule.parameters(), 'lr':lr} for x in range(pnet.number_of_pcoders)],
    weight_decay=5e-4)

# Set up TrainSoundsDataset

In [ ]:
train_dataset = TrainCleanSoundsDataset(train_datafile)

# Set up PsychophysicsSoundsDataset

In [ ]:
f_in = h5py.File(f"{engram_dir}PsychophysicsWord2017W_not_resampled.hdf5", 'r')

In [ ]:
f_metadata = np.load(f"{engram_dir}PsychophysicsWord2017W_999c6fc475be1e82e114ab9865aa5459e4fd329d.__META.npy", 'r')

In [ ]:
f_key = np.load(f"{engram_dir}PsychophysicsWord2017W_999c6fc475be1e82e114ab9865aa5459e4fd329d.__META_key.npy", 'r')

In [ ]:
def getPsychophysics2017WCleanCochleagrams():
    
    cochleagrams_clean = []
   
    cochleagrams = []
    for batch_ii in range(0,15300,100):
        hdf5_path = '/mnt/smb/locker/issa-locker/users/Erica/cgrams_for_noise_robustness_analysis/PsychophysicsWord2017W_clean/batch_'+str(batch_ii)+'_to_'+str(batch_ii+100)+'.hdf5'
        with h5py.File(hdf5_path, 'r') as f_in:
            cochleagrams += list(f_in['data'])

    return cochleagrams
clean_in = getPsychophysics2017WCleanCochleagrams()
clean_in = np.array(clean_in)

In [ ]:
labels = []
for word in f_metadata['word']:
    idx = np.argwhere(f_key == word)
    if len(idx) == 0:
        labels.append(-1)
    else:
        labels.append(idx.item())
labels = np.array(labels)
labels += 1

In [ ]:
orig_dset = []
for _orig_dset in f_metadata['orig_dset']:
    _orig_dset = str(_orig_dset, 'utf-8')
    _orig_dset = 'WSJ' if 'WSJ' in _orig_dset else 'Timit'
    orig_dset.append(_orig_dset)
orig_dset = np.array(orig_dset)

In [ ]:
psycho_dataset = PsychophysicsCleanSoundsDataset(
    clean_in, labels, orig_dset, exclude_timit=True
    )

In [ ]:
del orig_dset
del clean_in
del labels

# Merge Datasets

In [ ]:
full_dataset = CleanSoundsDataset(train_dataset, psycho_dataset)
del train_dataset
del psycho_dataset
n_train = int(len(full_dataset)*0.9)
train_dataset = Subset(full_dataset, np.arange(n_train))
eval_dataset = Subset(full_dataset, np.arange(n_train, len(full_dataset)))
train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True,
    num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY
    )
eval_loader = DataLoader(
    eval_dataset, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY
    )

torch.Size([55188, 1, 164, 400])


# Set up checkpoints and tensorboards

In [ ]:
checkpoint_path = os.path.join(checkpoints_dir, f"{pnet_name}")
if not os.path.exists(checkpoint_path):
    os.makedirs(checkpoint_path)
checkpoint_path = os.path.join(checkpoint_path, pnet_name + '-{epoch}-{type}.pth')

# summarywriter
from torch.utils.tensorboard import SummaryWriter
tensorboard_path = os.path.join(tensorboard_dir, f"{pnet_name}")
if not os.path.exists(tensorboard_path):
    os.makedirs(tensorboard_path)
sumwriter = SummaryWriter(tensorboard_path, filename_suffix=f'')

# Train

In [ ]:
loss_function = torch.nn.MSELoss()
for epoch in range(1, NUM_EPOCHS+1):
    train_pcoders(pnet, optimizer, loss_function, epoch, train_loader, DEVICE, sumwriter)
    eval_pcoders(pnet, loss_function, epoch, eval_loader, DEVICE, sumwriter)

    # save checkpoints every 5 epochs
    if epoch % 5 == 0:
        torch.save(pnet.state_dict(), checkpoint_path.format(epoch=epoch, type='regular'))

/home/es3773/.local/lib/python3.7/site-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


Training Epoch: 1 [64/49669]	Loss: 418.6643
Training Epoch: 1 [128/49669]	Loss: 5880.7471
Training Epoch: 1 [192/49669]	Loss: 763.0746
Training Epoch: 1 [256/49669]	Loss: 1562.0483
Training Epoch: 1 [320/49669]	Loss: 3871.1531
Training Epoch: 1 [384/49669]	Loss: 2819.1885
Training Epoch: 1 [448/49669]	Loss: 1059.6582
Training Epoch: 1 [512/49669]	Loss: 469.8101
Training Epoch: 1 [576/49669]	Loss: 1264.3372
Training Epoch: 1 [640/49669]	Loss: 2022.7345
Training Epoch: 1 [704/49669]	Loss: 1959.6693
Training Epoch: 1 [768/49669]	Loss: 1060.3384
Training Epoch: 1 [832/49669]	Loss: 499.5949
Training Epoch: 1 [896/49669]	Loss: 499.8237
Training Epoch: 1 [960/49669]	Loss: 933.9595
Training Epoch: 1 [1024/49669]	Loss: 1326.8002
Training Epoch: 1 [1088/49669]	Loss: 1120.5374
Training Epoch: 1 [1152/49669]	Loss: 715.4351
Training Epoch: 1 [1216/49669]	Loss: 467.3267
Training Epoch: 1 [1280/49669]	Loss: 478.2563
Training Epoch: 1 [1344/49669]	Loss: 671.1796
Training Epoch: 1 [1408/49669]	Loss: 90

Training Epoch: 1 [11456/49669]	Loss: 450.7382
Training Epoch: 1 [11520/49669]	Loss: 381.8687
Training Epoch: 1 [11584/49669]	Loss: 434.0299
Training Epoch: 1 [11648/49669]	Loss: 449.0807
Training Epoch: 1 [11712/49669]	Loss: 422.3297
Training Epoch: 1 [11776/49669]	Loss: 398.6059
Training Epoch: 1 [11840/49669]	Loss: 425.7851
Training Epoch: 1 [11904/49669]	Loss: 417.0107
Training Epoch: 1 [11968/49669]	Loss: 432.1522
Training Epoch: 1 [12032/49669]	Loss: 408.2046
Training Epoch: 1 [12096/49669]	Loss: 415.4198
Training Epoch: 1 [12160/49669]	Loss: 437.2926
Training Epoch: 1 [12224/49669]	Loss: 380.7126
Training Epoch: 1 [12288/49669]	Loss: 361.0434
Training Epoch: 1 [12352/49669]	Loss: 452.8088
Training Epoch: 1 [12416/49669]	Loss: 422.7009
Training Epoch: 1 [12480/49669]	Loss: 399.8064
Training Epoch: 1 [12544/49669]	Loss: 420.1967
Training Epoch: 1 [12608/49669]	Loss: 429.9666
Training Epoch: 1 [12672/49669]	Loss: 410.1173
Training Epoch: 1 [12736/49669]	Loss: 424.1634
Training Epoc

Training Epoch: 1 [22656/49669]	Loss: 426.0735
Training Epoch: 1 [22720/49669]	Loss: 407.8344
Training Epoch: 1 [22784/49669]	Loss: 425.2044
Training Epoch: 1 [22848/49669]	Loss: 433.7865
Training Epoch: 1 [22912/49669]	Loss: 412.3562
Training Epoch: 1 [22976/49669]	Loss: 410.2115
Training Epoch: 1 [23040/49669]	Loss: 439.9001
Training Epoch: 1 [23104/49669]	Loss: 382.5101
Training Epoch: 1 [23168/49669]	Loss: 440.5617
Training Epoch: 1 [23232/49669]	Loss: 432.9395
Training Epoch: 1 [23296/49669]	Loss: 435.9741
Training Epoch: 1 [23360/49669]	Loss: 394.8442
Training Epoch: 1 [23424/49669]	Loss: 441.6146
Training Epoch: 1 [23488/49669]	Loss: 434.8226
Training Epoch: 1 [23552/49669]	Loss: 444.3911
Training Epoch: 1 [23616/49669]	Loss: 405.8032
Training Epoch: 1 [23680/49669]	Loss: 418.3300
Training Epoch: 1 [23744/49669]	Loss: 411.5098
Training Epoch: 1 [23808/49669]	Loss: 418.2091
Training Epoch: 1 [23872/49669]	Loss: 420.5782
Training Epoch: 1 [23936/49669]	Loss: 435.7285
Training Epoc

Training Epoch: 1 [33856/49669]	Loss: 413.0791
Training Epoch: 1 [33920/49669]	Loss: 416.2403
Training Epoch: 1 [33984/49669]	Loss: 395.6244
Training Epoch: 1 [34048/49669]	Loss: 431.0198
Training Epoch: 1 [34112/49669]	Loss: 416.6628
Training Epoch: 1 [34176/49669]	Loss: 414.9441
Training Epoch: 1 [34240/49669]	Loss: 416.2034
Training Epoch: 1 [34304/49669]	Loss: 438.1298
Training Epoch: 1 [34368/49669]	Loss: 442.4864
Training Epoch: 1 [34432/49669]	Loss: 406.2890
Training Epoch: 1 [34496/49669]	Loss: 429.5029
Training Epoch: 1 [34560/49669]	Loss: 423.8179
Training Epoch: 1 [34624/49669]	Loss: 431.0344
Training Epoch: 1 [34688/49669]	Loss: 406.7956
Training Epoch: 1 [34752/49669]	Loss: 399.3762
Training Epoch: 1 [34816/49669]	Loss: 422.6410
Training Epoch: 1 [34880/49669]	Loss: 432.2469
Training Epoch: 1 [34944/49669]	Loss: 420.7932
Training Epoch: 1 [35008/49669]	Loss: 418.7831
Training Epoch: 1 [35072/49669]	Loss: 457.6133
Training Epoch: 1 [35136/49669]	Loss: 411.4021
Training Epoc

...

Training Epoch: 30 [7040/49669]	Loss: 398.6771
Training Epoch: 30 [7104/49669]	Loss: 383.9459
Training Epoch: 30 [7168/49669]	Loss: 406.2878
Training Epoch: 30 [7232/49669]	Loss: 386.9082
Training Epoch: 30 [7296/49669]	Loss: 395.3141
Training Epoch: 30 [7360/49669]	Loss: 383.7905
Training Epoch: 30 [7424/49669]	Loss: 409.9842
Training Epoch: 30 [7488/49669]	Loss: 411.0700
Training Epoch: 30 [7552/49669]	Loss: 398.1660
Training Epoch: 30 [7616/49669]	Loss: 385.6425
Training Epoch: 30 [7680/49669]	Loss: 436.2613
Training Epoch: 30 [7744/49669]	Loss: 440.4814
Training Epoch: 30 [7808/49669]	Loss: 388.0141
Training Epoch: 30 [7872/49669]	Loss: 402.6691
Training Epoch: 30 [7936/49669]	Loss: 373.1789
Training Epoch: 30 [8000/49669]	Loss: 382.3628
Training Epoch: 30 [8064/49669]	Loss: 399.0727
Training Epoch: 30 [8128/49669]	Loss: 412.3539
Training Epoch: 30 [8192/49669]	Loss: 394.3021
Training Epoch: 30 [8256/49669]	Loss: 388.0067
Training Epoch: 30 [8320/49669]	Loss: 410.6070
Training Epoc

Training Epoch: 30 [18048/49669]	Loss: 405.4565
Training Epoch: 30 [18112/49669]	Loss: 449.8830
Training Epoch: 30 [18176/49669]	Loss: 416.0983
Training Epoch: 30 [18240/49669]	Loss: 400.2983
Training Epoch: 30 [18304/49669]	Loss: 433.2831
Training Epoch: 30 [18368/49669]	Loss: 422.2506
Training Epoch: 30 [18432/49669]	Loss: 406.2612
Training Epoch: 30 [18496/49669]	Loss: 416.3469
Training Epoch: 30 [18560/49669]	Loss: 381.7301
Training Epoch: 30 [18624/49669]	Loss: 374.2618
Training Epoch: 30 [18688/49669]	Loss: 354.2135
Training Epoch: 30 [18752/49669]	Loss: 384.6910
Training Epoch: 30 [18816/49669]	Loss: 405.5122
Training Epoch: 30 [18880/49669]	Loss: 419.2879
Training Epoch: 30 [18944/49669]	Loss: 375.9311
Training Epoch: 30 [19008/49669]	Loss: 424.5264
Training Epoch: 30 [19072/49669]	Loss: 432.1579
Training Epoch: 30 [19136/49669]	Loss: 409.2379
Training Epoch: 30 [19200/49669]	Loss: 368.6528
Training Epoch: 30 [19264/49669]	Loss: 401.3951
Training Epoch: 30 [19328/49669]	Loss: 3

Training Epoch: 30 [28992/49669]	Loss: 403.7639
Training Epoch: 30 [29056/49669]	Loss: 368.4398
Training Epoch: 30 [29120/49669]	Loss: 407.1647
Training Epoch: 30 [29184/49669]	Loss: 396.9297
Training Epoch: 30 [29248/49669]	Loss: 363.5916
Training Epoch: 30 [29312/49669]	Loss: 345.5349
Training Epoch: 30 [29376/49669]	Loss: 401.1095
Training Epoch: 30 [29440/49669]	Loss: 423.1356
Training Epoch: 30 [29504/49669]	Loss: 406.6464
Training Epoch: 30 [29568/49669]	Loss: 382.6423
Training Epoch: 30 [29632/49669]	Loss: 397.7026
Training Epoch: 30 [29696/49669]	Loss: 404.7704
Training Epoch: 30 [29760/49669]	Loss: 411.0496
Training Epoch: 30 [29824/49669]	Loss: 434.8773
Training Epoch: 30 [29888/49669]	Loss: 444.2714
Training Epoch: 30 [29952/49669]	Loss: 386.3034
Training Epoch: 30 [30016/49669]	Loss: 392.7641
Training Epoch: 30 [30080/49669]	Loss: 387.7584
Training Epoch: 30 [30144/49669]	Loss: 397.8548
Training Epoch: 30 [30208/49669]	Loss: 384.3885
Training Epoch: 30 [30272/49669]	Loss: 4

Training Epoch: 30 [39936/49669]	Loss: 399.7203
Training Epoch: 30 [40000/49669]	Loss: 432.9738
Training Epoch: 30 [40064/49669]	Loss: 416.7090
Training Epoch: 30 [40128/49669]	Loss: 412.2441
Training Epoch: 30 [40192/49669]	Loss: 422.2906
Training Epoch: 30 [40256/49669]	Loss: 414.4973
Training Epoch: 30 [40320/49669]	Loss: 403.1668
Training Epoch: 30 [40384/49669]	Loss: 397.2641
Training Epoch: 30 [40448/49669]	Loss: 389.8711
Training Epoch: 30 [40512/49669]	Loss: 398.9217
Training Epoch: 30 [40576/49669]	Loss: 412.5717
Training Epoch: 30 [40640/49669]	Loss: 417.2870
Training Epoch: 30 [40704/49669]	Loss: 387.6442
Training Epoch: 30 [40768/49669]	Loss: 443.9510
Training Epoch: 30 [40832/49669]	Loss: 419.0638
Training Epoch: 30 [40896/49669]	Loss: 406.0201
Training Epoch: 30 [40960/49669]	Loss: 405.9333
Training Epoch: 30 [41024/49669]	Loss: 408.9439
Training Epoch: 30 [41088/49669]	Loss: 413.4149
Training Epoch: 30 [41152/49669]	Loss: 415.9998
Training Epoch: 30 [41216/49669]	Loss: 4

Training Epoch: 31 [1152/49669]	Loss: 425.2187
Training Epoch: 31 [1216/49669]	Loss: 505.1439
Training Epoch: 31 [1280/49669]	Loss: 443.9224
Training Epoch: 31 [1344/49669]	Loss: 410.7302
Training Epoch: 31 [1408/49669]	Loss: 468.8516
Training Epoch: 31 [1472/49669]	Loss: 441.6345
Training Epoch: 31 [1536/49669]	Loss: 416.0933
Training Epoch: 31 [1600/49669]	Loss: 428.9854
Training Epoch: 31 [1664/49669]	Loss: 418.7392
Training Epoch: 31 [1728/49669]	Loss: 415.9656
Training Epoch: 31 [1792/49669]	Loss: 463.5035
Training Epoch: 31 [1856/49669]	Loss: 430.1030
Training Epoch: 31 [1920/49669]	Loss: 370.4975
Training Epoch: 31 [1984/49669]	Loss: 408.7682
Training Epoch: 31 [2048/49669]	Loss: 392.2877
Training Epoch: 31 [2112/49669]	Loss: 416.8257
Training Epoch: 31 [2176/49669]	Loss: 402.4201
Training Epoch: 31 [2240/49669]	Loss: 410.8046
Training Epoch: 31 [2304/49669]	Loss: 433.4731
Training Epoch: 31 [2368/49669]	Loss: 428.3170
Training Epoch: 31 [2432/49669]	Loss: 399.6384
Training Epoc